## pitchとplayerの前処理を統合
#### 22A
- all_pitcher_17, all_player_17
- 22相当に戻す

#### 23
- all_pitcher_16, all_player_16
- 2017年の集計がない選手はNaNに

#### 22
- all_pitcher_15, all_player_15
- 2017年の結果を1/4ずつ集計

#### 21
- 19相当に戻す, 球種サブモデルの予測方法変更

#### 20
- player同士の組み合わせを削除

#### 19
- all_pitch_14, all_pitcher_14.f, all_player_14

#### 18
- all_pitch_13, all_pitcher_13.f, all_player_13

#### 17
- all_pitch_12

#### 16
- all_pitch_11, all_pitcher_12.f, all_player_12

#### 15
- all_pitch_10

#### 14
- all_pitch_9 ,all_player_11, all_pitcher_player_1.f

#### 13
- all_pitch_8 ,all_player_10

In [1]:
import pandas as pd
import numpy as np
import feather
pd.set_option('display.max_Columns', 300)

### 管理番号

In [2]:
model_No = '31'

### sub-modelを使用するかどうか

In [3]:
use_sub_model = False

### n/4

In [4]:
sample_No = 1

In [5]:
ALL_PITCH = 'intermediate/pitch/all_pitch.f'
ALL_PITCHER = 'intermediate/player/all_pitcher_{}.f'.format(sample_No)
ALL_CATCHER = 'intermediate/player/all_catcher_{}.f'.format(sample_No)
ALL_PLAYER = 'intermediate/player/all_player_{}.f'.format(sample_No)

SUB_BALL = 'intermediate/{}/ball_predict_{}_{}.f'.format(model_No, model_No, sample_No)
SUB_COURSE = 'intermediate/{}/course_predict_{}_{}.f'.format(model_No, model_No, sample_No)

OUTPUT = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)
OUTPUT_SUB = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)

In [6]:
all_pitch = pd.read_feather(ALL_PITCH)
all_pitch.shape

(778767, 119)

In [7]:
all_pitcher = pd.read_feather(ALL_PITCHER)
all_pitcher.shape

(2832, 55)

In [8]:
all_catcher = pd.read_feather(ALL_CATCHER)
all_catcher.shape

(480, 34)

In [9]:
all_player = pd.read_feather(ALL_PLAYER)
all_player.shape

(2757, 21)

In [11]:
all_catcher.head()

,選手ID,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball
0,11343,R_L,0.459320,0.081594,0.137698,0.082619,0.111789,0.055126,0.018639,0.053215,613.142857,0.061323,0.088211,0.069664,0.043989,0.057968,0.042218,0.033318,0.048555,0.036626,0.152423,0.063327,0.153821,0.148555,0.138630,0.215750,0.194734,0.148509,0.302377,0.219199,0.306244,0.144175,0.118500,0.211883
1,11343,R_R,0.451001,0.085375,0.214948,0.068821,0.085582,0.027976,0.005545,0.060752,690.400000,0.036294,0.045108,0.030417,0.050447,0.063524,0.043536,0.056903,0.083761,0.071098,0.097087,0.091831,0.070601,0.259394,0.143643,0.188917,0.192394,0.145050,0.329995,0.111819,0.167687,0.157507,0.211761,0.351225
2,11343,L_L,0.512595,0.051964,0.280254,0.051612,0.025366,0.030826,0.009688,0.037696,177.406250,0.068346,0.110974,0.094768,0.042980,0.060067,0.036463,0.025366,0.034173,0.027655,0.099524,0.089660,0.250661,0.059362,0.136692,0.189184,0.205214,0.158887,0.310023,0.274088,0.350185,0.139510,0.087194,0.149022
3,11343,L_R,0.460879,0.050418,0.166423,0.069142,0.048745,0.148326,0.025418,0.030649,318.666667,0.040481,0.057950,0.053766,0.039644,0.055962,0.044142,0.047490,0.063703,0.054498,0.073222,0.125000,0.174791,0.169351,0.127615,0.198222,0.177615,0.152406,0.344142,0.152197,0.248013,0.139749,0.165690,0.294351
4,11726,R_L,0.432432,0.054054,0.270270,0.027027,0.162162,0.000000,0.000000,0.054054,37.000000,0.081081,0.081081,0.054054,0.054054,0.081081,0.081081,0.000000,0.027027,0.027027,0.189189,0.000000,0.135135,0.189189,0.135135,0.189189,0.189189,0.162162,0.324324,0.216216,0.324324,0.216216,0.054054,0.189189


### Join
- pitch.投手ID/年度 - player.選手ID/年度
- pitch.打者ID/年度 - player.選手ID/年度
- pitch.捕手ID/年度 - player.選手ID/年度

In [193]:
merge_all = pd.merge(all_pitch, all_pitcher, left_on=['投手ID', '年度', 'pit_bat'], right_on=['選手ID', '年度', 'pit_bat'], how='left')

In [194]:
merge_all = pd.merge(merge_all, all_player, left_on=['打者ID', '年度'], right_on=['選手ID', '年度'], how='left', suffixes=['_pit', '_bat'])

In [195]:
merge_all = pd.merge(merge_all, all_player.drop(columns=['batter_cnt', 'bat_game_cnt']), left_on=['捕手ID', '年度'], right_on=['選手ID', '年度'], how='left')

### player同士の組み合わせ
- 年棒の差
- 現役年数の差
- 年齢の差
- salary_yearの差
- salary_x_yearの差
- BMIの差

In [196]:
merge_all['salary_dif_p-b'] = merge_all['salary_pit'] - merge_all['salary_bat']
merge_all['play_year_dif_p-b'] = merge_all['play_year_pit'] - merge_all['play_year_bat']
merge_all['age_dif_p-b'] = merge_all['age_pit'] - merge_all['age_bat']
merge_all['salary_year_dif_p-b'] = merge_all['salary_year_pit'] - merge_all['salary_year_bat']
merge_all['salary_x_year_dif_p-b'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year_bat']
merge_all['rank_year_dif_p-b'] = merge_all['rank_year_pit'] - merge_all['rank_year_bat']
merge_all['rank_x_year_dif_p-b'] = merge_all['rank_x_year_pit'] - merge_all['rank_x_year_bat']
merge_all['bmi_dif_p-b'] = merge_all['bmi_pit'] - merge_all['bmi_bat']

merge_all['salary_dif_p-c'] = merge_all['salary_pit'] - merge_all['salary']
merge_all['play_year_dif_p-c'] = merge_all['play_year_pit'] - merge_all['play_year']
merge_all['age_dif_p-c'] = merge_all['age_pit'] - merge_all['age']
merge_all['salary_year_dif_p-c'] = merge_all['salary_year_pit'] - merge_all['salary_year']
merge_all['salary_x_year_dif_p-c'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year']
merge_all['rank_year_dif_p-c'] = merge_all['rank_year_pit'] - merge_all['rank_year']
merge_all['rank_x_year_dif_p-c'] = merge_all['rank_x_year_pit'] - merge_all['rank_x_year']
merge_all['bmi_dif_p-c'] = merge_all['bmi_pit'] - merge_all['bmi']

merge_all['salary_dif_b-c'] = merge_all['salary_bat'] - merge_all['salary']
merge_all['play_year_dif_b-c'] = merge_all['play_year_bat'] - merge_all['play_year']
merge_all['age_dif_b-c'] = merge_all['age_bat'] - merge_all['age']
merge_all['salary_year_dif_b-c'] = merge_all['salary_year_bat'] - merge_all['salary_year']
merge_all['salary_x_year_dif_b-c'] = merge_all['salary_x_year_bat'] - merge_all['salary_x_year']
merge_all['rank_year_dif_b-c'] = merge_all['rank_year_bat'] - merge_all['rank_year']
merge_all['rank_x_year_dif_b-c'] = merge_all['rank_x_year_bat'] - merge_all['rank_x_year']
merge_all['bmi_dif_b-c'] = merge_all['bmi_bat'] - merge_all['bmi']

### 球種の組合せ

In [197]:
ball_kind = ['straight', 'curve', 'slider', 'shoot', 'fork', 'changeup', 'sinker', 'cutball']

In [198]:
for ball in ball_kind:
    target = 'sub_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] - merge_all[ball]

In [199]:
for ball in ball_kind:
    target = 'div_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] / merge_all[ball]

In [200]:
for ball in ball_kind:
    target = 'mul_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] * merge_all[ball]

In [201]:
for ball in ball_kind:
    target = 'ave_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = (merge_all[bc_src] + merge_all[ball])/2

In [202]:
for ball in ball_kind:
    target = 'rate_' + ball
    ave_src = 'ave_' + ball
    merge_all[target] = merge_all[ave_src] /(1-merge_all['ave_straight'])

In [203]:
ball_kind_bc = list(map(lambda x: 'bc_' + x, ball_kind))
merge_all.drop(columns=ball_kind, inplace=True)
merge_all.drop(columns=ball_kind_bc, inplace=True)

### コースの組合せ

In [204]:
course_kind = ['course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 
                'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12',
               'high_str', 'high_ball', 'mid_str', 'low_str', 'low_ball', 
               'left_str', 'left_ball', 'center_str', 'right_str', 'right_ball']

In [205]:
for course in course_kind:
    target = 'sub_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] - merge_all[course]

In [206]:
for course in course_kind:
    target = 'div_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] / merge_all[course]

In [207]:
for course in course_kind:
    target = 'mul_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] * merge_all[course]

In [208]:
for course in course_kind:
    target = 'ave_' + course
    bc_src = 'bc_' + course
    merge_all[target] = (merge_all[bc_src] + merge_all[course])/2

In [209]:
course_kind_bc = list(map(lambda x: 'bc_' + x, course_kind))
merge_all.drop(columns=course_kind, inplace=True)
merge_all.drop(columns=course_kind_bc, inplace=True)

In [210]:
merge_all.head()

,データ内連番,ball,course,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,選手ID_pit,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,選手ID_bat,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,...,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,sub_straight,sub_curve,sub_slider,sub_shoot,sub_fork,sub_changeup,sub_sinker,sub_cutball,div_straight,div_curve,div_slider,div_shoot,div_fork,div_changeup,div_sinker,div_cutball,mul_straight,mul_curve,mul_slider,mul_shoot,mul_fork,mul_changeup,mul_sinker,mul_cutball,ave_straight,ave_curve,ave_slider,ave_shoot,ave_fork,ave_changeup,ave_sinker,ave_cutball,rate_straight,rate_curve,rate_slider,rate_shoot,rate_fork,rate_changeup,rate_sinker,rate_cutball,sub_course_0,sub_course_1,sub_course_2,sub_course_3,sub_course_4,sub_course_5,sub_course_6,sub_course_7,sub_course_8,sub_course_9,sub_course_10,sub_course_11,sub_course_12,sub_high_str,sub_high_ball,sub_mid_str,sub_low_str,sub_low_ball,sub_left_str,sub_left_ball,sub_center_str,sub_right_str,sub_right_ball,div_course_0,div_course_1,div_course_2,div_course_3,div_course_4,div_course_5,div_course_6,div_course_7,div_course_8,div_course_9,div_course_10,div_course_11,div_course_12,div_high_str,div_high_ball,div_mid_str,div_low_str,div_low_ball,div_left_str,div_left_ball,div_center_str,div_right_str,div_right_ball,mul_course_0,mul_course_1,mul_course_2,mul_course_3,mul_course_4,mul_course_5,mul_course_6,mul_course_7,mul_course_8,mul_course_9,mul_course_10,mul_course_11,mul_course_12,mul_high_str,mul_high_ball,mul_mid_str,mul_low_str,mul_low_ball,mul_left_str,mul_left_ball,mul_center_str,mul_right_str,mul_right_ball,ave_course_0,ave_course_1,ave_course_2,ave_course_3,ave_course_4,ave_course_5,ave_course_6,ave_course_7,ave_course_8,ave_course_9,ave_course_10,ave_course_11,ave_course_12,ave_high_str,ave_high_ball,ave_mid_str,ave_low_str,ave_low_ball,ave_left_str,ave_left_ball,ave_center_str,ave_right_str,ave_right_ball
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0,0,0,0,0,1,0,2017-03-31,2017-03-31,0,2020-05-31 18:03:00,2020-05-31 18:03:00,00:00:00,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,40

### ダミー変数

In [207]:
merge_all = pd.get_dummies(merge_all, columns=['pit_bat'])

### 不要な列を削除

In [208]:
merge_all.drop(columns=[
    '選手ID_pit', '選手ID_bat', '選手ID',
    '試合内連番',
    '年度', 
    '試合ID', 
    'ホームチームID', 'アウェイチームID', 
    '投手ID', '投手チームID', 
    '打者ID', '打者チームID', 
    'プレイ前走者状況', 
    '捕手ID', 
    'opening_date', 'game_date',
    'start_time', 'game_time', 'elapsed_time'
], inplace=True)

### Rename

In [209]:
merge_all.rename(columns={
    'データ内連番': 'No',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [210]:
print(merge_all.shape)
merge_all.head()

(778767, 190)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-

### 出力(sub-modelなし)

In [211]:
if not use_sub_model:
    merge_all.to_feather(OUTPUT)
    print(OUTPUT)

### 予測結果を特徴量に加える

In [212]:
if use_sub_model:
    sub_ball = pd.read_feather(SUB_BALL)
    sub_course = pd.read_feather(SUB_COURSE)
    print(sub_ball.shape)
    print(sub_course.shape)

(701651, 8)
(701651, 13)


In [213]:
if use_sub_model:
    merge_sub = pd.concat([merge_all, sub_ball], axis=1)
    merge_sub = pd.concat([merge_sub, sub_course], axis=1)
    print(merge_sub.shape)

(778767, 211)


### 出力(sub-modelあり)

In [214]:
if use_sub_model:
    merge_sub.to_feather(OUTPUT_SUB)
    print(OUTPUT_SUB)

intermediate/22A/all_merge_22A_4_sub.f


### GCS
#### local->GCS

In [215]:
GCS = 'gs://yasunobu_baseball/'

In [216]:
'gsutil cp ./{} {}intermediate/'.format(OUTPUT, GCS)

'gsutil cp ./intermediate/22A/all_merge_22A_4.f gs://yasunobu_baseball/intermediate/'

In [217]:
'gsutil cp ./{} {}intermediate/'.format(OUTPUT_SUB, GCS)

'gsutil cp ./intermediate/22A/all_merge_22A_4_sub.f gs://yasunobu_baseball/intermediate/'

#### GCS->GCE

In [218]:
'gsutil cp {}{} ~/work/baseball/intermediate'.format(GCS, OUTPUT)

'gsutil cp gs://yasunobu_baseball/intermediate/22A/all_merge_22A_4.f ~/work/baseball/intermediate'